In [1]:
import pandas as pd
import re

In [2]:
instructions = {
    'ADD': '18',
    'AND': '40',
    'COMP': '28',
    'DIV': '24',
    'J': '3C',
    'JEQ': '30',
    'JGT': '34',
    'JLT': '38',
    'JSUB': '48',
    'LDA': '00',
    'LDCH': '50',
    'LDL': '08',
    'LDX': '04',
    'MUL': '20',
    'OR': '44',
    'RD': 'D8',
    'RSUB': '4C',
    'STA': '0C',
    'STCH': '54',
    'STL': '14',
    'STSW': 'E8',
    'STX': '10',
    'SUB': '1C',
    'TD': 'E0',
    'TIX': '2C',
    'WD': 'DC',
    'FIX': ['C4', 1],
    'FLOAT': ['CO', 1],
    'HIO': ['F4', 1],
    'NORM': ['C8', 1],
    'SIO': ['F0', 1],
    'TIO': ['F8', 1],
}

directives = ['START', 'END', 'BYTE', 'WORD', 'RESB', 'RESW']

In [3]:
def is_instruction_directive(string):
    return string in instructions or string in directives

In [4]:
def open_file(file):
    with open(file) as f:
        program = [re.sub(r'\s+', ' ', line).strip().upper() for line in f]
    return program

In [5]:
def helper_1():
    fout = open("zeee.txt", "w", encoding='utf-8')
    for ind in df.index:
        if(df.Label[ind] == ' '):
            fout.write('\t\t{0}\t{1}\n'.format(df.Mnemonic[ind].ljust(8, ' '), df.Value[ind]).ljust(8, ' '))

        else:
            fout.write('{0}\t{1}\t{2}\n'.format(df.Label[ind].ljust(8, ' '), df.Mnemonic[ind].ljust(8, ' '), df.Value[ind].ljust(8, ' ')))
    fout.close()
    return

In [6]:
def prog_name(df):
    return df.iloc[0].Label.ljust(6, 'x')

In [7]:
def return_df(program):
    dict = []
    for line in program:
        
        temp = line.split(" ")[1: 4]
        if len(temp) < 1:
            continue
        if temp[0] == '.':
            continue
        if temp[0] in instructions:
            temp = temp[: 2] 
        print(temp)
        if not is_instruction_directive(temp[0]) and not is_instruction_directive(temp[1]):
            print('{} or {} is not found'.format(temp[0], temp[1]))
            return 
            #quit()
        
        if temp[0]  == 'RSUB':
            label = ' '
            mnemonic = temp[0]
            value = ' '
            
        elif temp[0]  == 'END':
            label = ' '
            mnemonic = temp[0]
            value = temp[1]

        elif temp[1] == 'RSUB':
            label = temp[0]
            mnemonic = temp[1]
            value = ' '
        
            
        elif temp[0] in instructions:
            label = ' '
            mnemonic = temp[0]
            value = temp[1]

        elif len(temp) >= 3:
            label = temp[0]
            mnemonic = temp[1]
            value = temp[2]
        

        dict.append({
            'Label':label,
            'Mnemonic': mnemonic,
            'Value': value
        })
    df = pd.DataFrame(dict)

    if df.iloc[df.index.stop -1].Mnemonic != 'END':
        df2 = pd.DataFrame([[' ', 'END', df.iloc[0].Value]], columns=df.columns)
        df = pd.concat([df, df2], ignore_index = True)
    return df

In [8]:
program = open_file('in4.txt')
df = return_df(program)
df

['PROG1', 'START', '0000']
['LDA', 'ZERO']
['STA', 'INDEX']
['LOOP', 'LDX', 'INDEX']
['LDA', 'ZERO']
['STA', 'ALPHA,X']
['LDA', 'INDEX']
['ADD', 'THREE']
['STA', 'INDEX']
['COMP', 'K300']
['TIX', 'TWENTY']
['JLT', 'LOOP']
['INDEX', 'RESW', '1']
['ALPHA', 'RESW', '100']
['ZERO', 'WORD', '0']
['K300', 'WORD', '100']
['THREE', 'WORD', '3']
['TWENTY', 'WORD', '20']
['END', '0000']


,Label,Mnemonic,Value
0,PROG1,START,0000
1,,LDA,ZERO
2,,STA,INDEX
3,LOOP,LDX,INDEX
4,,LDA,ZERO
5,,STA,"ALPHA,X"
6,,LDA,INDEX
7,,ADD,THREE
8,,STA,INDEX
9,,COMP,K300


In [9]:

def location_counter(df):
    start_index = df.Value[0]
    list_counter = [" ", start_index]
    counter = start_index
    for i in range(1, df.index.stop):

        if df.Mnemonic[i] in instructions:
            if type(instructions[df.Mnemonic[i]]) == list:
                temp = hex(int(counter, 16) + 1)
            else:
                temp = hex(int(counter, 16) + 3)
            
        if df.Mnemonic[i] == 'WORD':
            temp = hex(int(counter, 16) + 3)
        if df.Mnemonic[i] == 'BYTE':
            value = df.Value[i].split('\'')
            length = len(value[1])
            if value[0] == 'C':
                temp = hex(int(counter, 16) + int(length))
            else:
                temp = hex(int(counter, 16) + int(int(length)/2))
                
        if df.Mnemonic[i] == 'RESW':
            temp = hex(int(counter, 16) + int(df.Value[i]) *3)
            
        if df.Mnemonic[i] == 'RESB':
            temp = hex(int(counter, 16) + int(df.Value[i]))
            
    
        counter = temp.split('x')[1].rjust(4, '0').upper()
        list_counter.append(counter)
        
    df.insert(0, 'Location_Counter', list_counter[:-1])
    return df

In [10]:
df = location_counter(df)

In [11]:
df

,Location_Counter,Label,Mnemonic,Value
0,,PROG1,START,0000
1,0000,,LDA,ZERO
2,0003,,STA,INDEX
3,0006,LOOP,LDX,INDEX
4,0009,,LDA,ZERO
5,000C,,STA,"ALPHA,X"
6,000F,,LDA,INDEX
7,0012,,ADD,THREE
8,0015,,STA,INDEX
9,0018,,COMP,K300


In [12]:
def symbol_table(df):
    list = {}
    # Ignoring the first row 
    for i in range(1, df.index.stop):
        if df.Label[i] != ' ':
            list[df.Label[i]] = df.Location_Counter[i]
    return list
sym_table = symbol_table(df)

In [13]:
sym_table

{'LOOP': '0006',
 'INDEX': '0021',
 'ALPHA': '0024',
 'ZERO': '0150',
 'K300': '0153',
 'THREE': '0156',
 'TWENTY': '0159'}

In [14]:
def object_code(df, sym_table):
    object_code_list = [' ']
    for i in range(1, df.index.stop):
        if df.Mnemonic[i] in instructions:
            if df.Mnemonic[i] == 'RSUB':
                object_code = "4C0000"
                
            elif type(instructions[df.Mnemonic[i]]) != list :
                op_code = bin(int(instructions[df.Mnemonic[i]], 16))[2: -1].rjust(7, '0')
                op_code += '1' if df.Value[i][0] == '#'  else  '0'
                
                op_code_index = op_code
                op_code_index += '1' if len(df.Value[i].split(',')) > 1 else  '0'
                
                if df.Value[i].split(',')[0] not in sym_table:
                    print('VARIABLE {0} DOES NOT EXIST'.format(df.Value[i].split(',')[0]))
                    return
                    #quit()
                
                address = sym_table[df.Value[i].split(',')[0]]
                address_binary = bin(int(address, 16))[2: ].rjust(15, '0')
                object_code = op_code_index + address_binary
                
                object_code = hex(int(object_code, 2))[2: ].rjust(6, '0').upper()
                
        elif df.Mnemonic[i] == 'WORD':
            object_code = hex(int(df.Value[i]))[2: ].rjust(6, '0').upper()
        
        elif df.Mnemonic[i] == 'BYTE':
            if df.Value[i].split('\'')[0] == 'X':
                object_code = df.Value[i].split('\'')[1].upper()
            else:
                string = df.Value[i].split('\'')[1]
                object_code = ''
                for ascii_value in string.encode('ascii'):
                    object_code += str(ascii_value)
        else:
            object_code = ' '
        object_code_list.append(object_code)
        
    df.insert(4, 'Object_code', object_code_list)   
    return df

In [15]:
df = object_code(df, sym_table)
df

,Location_Counter,Label,Mnemonic,Value,Object_code
0,,PROG1,START,0000,
1,0000,,LDA,ZERO,000150
2,0003,,STA,INDEX,0C0021
3,0006,LOOP,LDX,INDEX,040021
4,0009,,LDA,ZERO,000150
5,000C,,STA,"ALPHA,X",0C8024
6,000F,,LDA,INDEX,000021
7,0012,,ADD,THREE,180156
8,0015,,STA,INDEX,0C0021
9,0018,,COMP,K300,280153


In [16]:
def byte_counter(df):
    byte_list = ['0']
    for i in range(1, df.index.stop):
        if df.Mnemonic[i] == 'WORD':
            byte_count = '3'
            
        elif df.Mnemonic[i] == 'BYTE':
            if df.Value[i].split('\'')[0] == 'X':
                byte_count = int(len(df.Value[i].split('\'')[1])/2)
            else:
                byte_count = len(df.Value[i].split('\'')[1])
                
        elif df.Mnemonic[i] in instructions:
            if type(df.Mnemonic[i]) == list:
                byte_count = '1'
            else:
                byte_count = '3'
        else:
            byte_count = '0'
        byte_list.append(byte_count)
    df.insert(5, 'Bytes', byte_list) 
    return df

In [17]:
df = byte_counter(df)
df

,Location_Counter,Label,Mnemonic,Value,Object_code,Bytes
0,,PROG1,START,0000,,0
1,0000,,LDA,ZERO,000150,3
2,0003,,STA,INDEX,0C0021,3
3,0006,LOOP,LDX,INDEX,040021,3
4,0009,,LDA,ZERO,000150,3
5,000C,,STA,"ALPHA,X",0C8024,3
6,000F,,LDA,INDEX,000021,3
7,0012,,ADD,THREE,180156,3
8,0015,,STA,INDEX,0C0021,3
9,0018,,COMP,K300,280153,3


In [18]:
df

,Location_Counter,Label,Mnemonic,Value,Object_code,Bytes
0,,PROG1,START,0000,,0
1,0000,,LDA,ZERO,000150,3
2,0003,,STA,INDEX,0C0021,3
3,0006,LOOP,LDX,INDEX,040021,3
4,0009,,LDA,ZERO,000150,3
5,000C,,STA,"ALPHA,X",0C8024,3
6,000F,,LDA,INDEX,000021,3
7,0012,,ADD,THREE,180156,3
8,0015,,STA,INDEX,0C0021,3
9,0018,,COMP,K300,280153,3


In [54]:
def head(df):
    program_name = prog_name(df)
    start = df.iloc[1].Location_Counter.rjust(6, '0')
    end = df.iloc[df.index.stop -1].Location_Counter.rjust(6, '0')
    length = hex(int(end, 16) - int(start, 16))[2: ].rjust(6, '0').upper()
    return 'H'+program_name+start+length

In [55]:
def end(df):
    start = df.iloc[1].Location_Counter.rjust(6, '0')
    return 'E'+start

In [56]:
def text_helper(start, df):
    address = df.iloc[start].Location_Counter.rjust(6, '0')
    counter = 0
    index = start
    while(counter + int(df.iloc[index].Bytes) <= 30):
        if df.iloc[index].Bytes == '0':
            break
            
        counter += int(df.iloc[index].Bytes)
        index += 1
        
    end = df.iloc[index].Location_Counter
    length = hex(int(end, 16) - int(address, 16))[2: ].rjust(2, '0').upper()
    
    object_codes = ''
    for i in range(start, index):
        object_codes += df.iloc[i].Object_code
    return 'T' + address + length + object_codes, index

In [57]:
def text_generator(df):
    texts = []
    i = 0
    while(i < df.index.stop):
        if df.iloc[i].Bytes == '0':
            i += 1
        else:
            text, i = text_helper(i, df)
            texts.append(text)
    return texts

In [58]:
text = text_generator(df)
print(head(df))
for i in text:
    print(i)
print(end(df))

HPROG1x00000000015C
T0000001E0001500C00210400210001500C80240000211801560C00212801532C0159
T00001E03380006
T0001500C000000000064000003000014
E000000


In [52]:
df

,Location_Counter,Label,Mnemonic,Value,Object_code,Bytes
0,,PROG1,START,0000,,0
1,0000,,LDA,ZERO,000150,3
2,0003,,STA,INDEX,0C0021,3
3,0006,LOOP,LDX,INDEX,040021,3
4,0009,,LDA,ZERO,000150,3
5,000C,,STA,"ALPHA,X",0C8024,3
6,000F,,LDA,INDEX,000021,3
7,0012,,ADD,THREE,180156,3
8,0015,,STA,INDEX,0C0021,3
9,0018,,COMP,K300,280153,3
